# Extract BQ table in GCS.

## Args:
- bq_client_project_id (str): project id that will be used by the bq client
- source_project_id (str): project id from where BQ table will be extracted
- dataset_id (str): dataset id from where BQ table will be extracted
- table_name (str): table name (without project id and dataset id)
- dataset (str): output dataset artifact generated by the operation, this parameter will be passed automatically by the orchestrator
- dataset_location (str): bq dataset location. Defaults to "EU".
- extract_job_config (dict): dict containing optional parameters required by the bq extract operation. Defaults to None. See available parameters here
https://googleapis.dev/python/bigquery/latest/generated/google.cloud.bigqueryjobExtractJobConfig.html
- file_pattern (str): Exporting data into one or more files. If empty, then table data is exported to a single file. For multiple files and allowed pattern, see: https://cloud.google.com/bigquery/docs/exporting-data#exporting_data_into_one_or_more_files

## Outputs:
- dataset_directory (str): Output dataset directory
- dataset_uri (str): The dataset's GCS uri.

In [ ]:
from kfp.v2.dsl import Output, Dataset

import logging
import os
from google.cloud.exceptions import GoogleCloudError
from google.cloud import bigquery

In [ ]:
bq_client_project_id: str = ""
source_project_id: str = ""
dataset_id: str = ""
table_name: str = ""
dataset_location: str = "EU"
extract_job_config: dict = {}
file_pattern: str = ""
dataset_uri: str = ""

In [ ]:
full_table_id = f"{source_project_id}.{dataset_id}.{table_name}"

if extract_job_config is None:
    extract_job_config = {}

table = bigquery.table.Table(table_ref=full_table_id)
job_config = bigquery.job.ExtractJobConfig(**extract_job_config)
client = bigquery.client.Client(
    project=bq_client_project_id, location=dataset_location
)

# if file_pattern is provided, join dataset.uri with file_pattern
if file_pattern:
    dataset_uri = os.path.join(dataset.uri, file_pattern)
dataset_directory = os.path.dirname(dataset_uri)

logging.info(f"Extract table {table} to {dataset_uri}")
extract_job = client.extract_table(
    table,
    dataset_uri,
    job_config=job_config,
)

try:
    result = extract_job.result()
    logging.info("Table extracted, result: {}".format(result))
except GoogleCloudError as e:
    logging.error(e)
    logging.error(extract_job.error_result)
    logging.error(extract_job.errors)
    raise e

In [ ]:
import scrapbook
scrapbook.glue("dataset_directory", dataset_directory)
scrapbook.glue("dataset_uri", dataset_uri)